Here, we are woring with 2 large DataFrame transactions which contains billions of records including,user_id,txn_amount,txn_date. where, user contains user profile info with user_id as a key. yo we noticed that joins are taking forever or failing due to data skew  because a user_id areextremely frequent.So, our task is join both DataFrame on user_id and implement skew join optimization to handle this issue.

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,rand,expr

In [33]:
spark=SparkSession.builder.appName("skewedjoinhandling").getOrCreate()

Simulate skewed user data

In [23]:
# transactions=spark.read.csv("transections.csv",header=True,inferSchema=True)
# users=spark.read.csv("User_Data.csv",header=True,inferSchema=True)

In [ ]:
transactions=spark.read.parquet("("gs://your-bucket/transactions/")")
users=spark.read.parquet("gs://your-bucket/users/")

To check columns

In [34]:
transactions.columns

Add a salt key to transactions(replicate skewed keys)

In [26]:
transactions=transactions.withColumn("salt",(rand()*10).cast("int"))

Replicate users data fro skewed users_ids with multiple salt values(0-9)


In [27]:
user_salts=users.crossJoin(spark.range(10).withColumnRenamed("id","salt"))

join both user_id and salt

In [35]:
result=transactions.join(user_salts,on=["trans_id","salt"],how="inner")


In [ ]:
result.select("user_id","txn_amount","txn_date").show()